In [1]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [2]:
!sudo apt-get install -qq w3m # to act as web browser
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 122405 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


In [ ]:
!nvidia-smi

Tue Apr 25 16:32:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install transformers
!pip install simplet5
!pip install --upgrade transformers
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.1
    Uninstalling transformers-4.28.1:
      Successfully uninstalled transformers-4.28.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.2
    Uninstalling transformers-4.16.2:
      Successfully uninstalled transformers-4.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simplet5 0.1.4 requires transformers==4.16.2, bu

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow_hub as hub
import evaluate
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import SimpleT5
import os
import spacy
import tensorflow as tf

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [3]:
def KFolds(df, kf):
  NumberOfRows = len(df)
  Range = NumberOfRows // kf
  Test = []
  Train = []
  for i in range(kf):
    Start = i * Range
    End = (i+1)*Range
    Test.append(df.iloc[Start:End])
    Train.append(df.drop(df.index[Start:End]))
  return Train, Test
def cosine_similarity(x, y):
    # Ensure length of x and y are the same
    if len(x) != len(y):
        return None
    # Compute the dot product between x and y
    dot_product = np.dot(x, y)
    # Compute the L2 norms (magnitudes) of x and y
    magnitude_x = np.sqrt(np.sum(x**2))
    magnitude_y = np.sqrt(np.sum(y**2))
    # Compute the cosine similarity
    cosine_similarity = dot_product / (magnitude_x * magnitude_y)
    return cosine_similarity
def sim(u, v):
    return 1-(cosine_similarity(u, v)/np.pi)
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)
rouge = evaluate.load('rouge')
def AESscore(Predict=["The quick brown fox jumps over the lazy dog."], Actual=["I am a sentence for which I would like to get its embedding"]):
    total = 0
    count = 0
    for i in range(len(Predict)): # tf.constant([Predict[i], Actual[i]])
        embeddings = embed(tf.constant([Predict[i], Actual[i]]))
        total += sim(embeddings[0], embeddings[1])
        count += 1
    return (total/count)
def Rouphscore(Predict=["The quick brown fox jumps over the lazy dog."], Actual=["I am a sentence for which I would like to get its embedding"]):
    return rouge.compute(predictions=Predict, references=Actual)
def GetBLEUScore(Predict, Actual):
    predict = Predict.split()
    actual = Actual.split()
    smoothie = nltk.translate.bleu_score.SmoothingFunction().method4
    return nltk.translate.bleu_score.sentence_bleu([actual], predict, smoothing_function=smoothie)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Research Paper/Sample.csv")#, delimiter='~'
End = int(5000 * 0.3)
Test,Train = df.iloc[0:End], df.drop(df.index[0:End])
Path = "/content/drive/MyDrive/Research Paper/First_Train_sample_70-30"
model = SimpleT5()
model.load_model("t5","snrspeaks/t5-one-line-summary") # snrspeaks/t5-one-line-summary
model.train(train_df=Train,
            eval_df=Test,
            source_max_token_len=512,
            target_max_token_len=50,
            outputdir = Path + "/Kfold"+str(0)+"/Output",
            batch_size=8, max_epochs=15, use_gpu=True)
model.model.save_pretrained(Path + "/Kfold"+str(0)+"/SavedModel")
model.tokenizer.save_pretrained(Path + "/Kfold"+str(0)+"/SavedModel")
